In [1]:
import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "grid_search"
args.jigsaw_n_classes = 31
args.classify_only_sane = True
args.train_all = True
# args.jig_weight = 0.9
# args.bias_whole_image = 0.8
args.TTA = False
args.suffix = "check"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jig_weight in [0.0]: 
    for args.bias_whole_image in [0.6]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/grid_search/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0_TAll_bias0.6_classifyOnlySane_check_252
New epoch - lr: 0.001
0/55 of epoch 1/30 jigsaw : 3.474923, class : 1.957479 - acc jigsaw : 0.023438, class : 0.148438 [bs:128]
30/55 of epoch 1/30 jigsaw : 3.431078, class : 0.543424 - acc jigsaw : 0.062500, class : 0.765625 [bs:128]
Accuracies on val: jigsaw : 0.027743, class : 0.866330
Accuracies on test: jigsaw : 0.019531, class : 0.618164
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 3.483318, class : 0.462169 - acc jigsaw : 0.023438, class : 0.703125 [bs:128]
30/55 of epoch 2/30 jigsaw : 3.435907, class : 0.452515 - acc jigsaw : 0.031250, class : 0.726562 [bs:128]
Accuracies on val: jigsaw : 0.049180, class : 0.886507
Accuracies on te

Accuracies on val: jigsaw : 0.081967, class : 0.944515
Accuracies on test: jigsaw : 0.043945, class : 0.677246
New epoch - lr: 0.001
0/55 of epoch 24/30 jigsaw : 3.417873, class : 0.012282 - acc jigsaw : 0.070312, class : 0.914062 [bs:128]
30/55 of epoch 24/30 jigsaw : 3.464332, class : 0.024565 - acc jigsaw : 0.046875, class : 0.859375 [bs:128]
Accuracies on val: jigsaw : 0.065574, class : 0.939470
Accuracies on test: jigsaw : 0.035156, class : 0.666992
New epoch - lr: 0.0001
0/55 of epoch 25/30 jigsaw : 3.441877, class : 0.056866 - acc jigsaw : 0.039062, class : 0.804688 [bs:128]
30/55 of epoch 25/30 jigsaw : 3.451150, class : 0.017929 - acc jigsaw : 0.078125, class : 0.851562 [bs:128]
Accuracies on val: jigsaw : 0.054224, class : 0.940731
Accuracies on test: jigsaw : 0.034180, class : 0.656738
New epoch - lr: 0.0001
0/55 of epoch 26/30 jigsaw : 3.440557, class : 0.005771 - acc jigsaw : 0.015625, class : 0.796875 [bs:128]
30/55 of epoch 26/30 jigsaw : 3.424447, class : 0.013441 - acc

0/55 of epoch 16/30 jigsaw : 3.545681, class : 0.108487 - acc jigsaw : 0.023438, class : 0.765625 [bs:128]
30/55 of epoch 16/30 jigsaw : 3.592794, class : 0.040399 - acc jigsaw : 0.000000, class : 0.820312 [bs:128]
Accuracies on val: jigsaw : 0.005044, class : 0.929382
Accuracies on test: jigsaw : 0.005371, class : 0.675781
New epoch - lr: 0.001
0/55 of epoch 17/30 jigsaw : 3.591561, class : 0.071752 - acc jigsaw : 0.015625, class : 0.773438 [bs:128]
30/55 of epoch 17/30 jigsaw : 3.556364, class : 0.017309 - acc jigsaw : 0.023438, class : 0.867188 [bs:128]
Accuracies on val: jigsaw : 0.005044, class : 0.933165
Accuracies on test: jigsaw : 0.006836, class : 0.661621
New epoch - lr: 0.001
0/55 of epoch 18/30 jigsaw : 3.551482, class : 0.028606 - acc jigsaw : 0.015625, class : 0.820312 [bs:128]
30/55 of epoch 18/30 jigsaw : 3.546455, class : 0.150302 - acc jigsaw : 0.023438, class : 0.835938 [bs:128]
Accuracies on val: jigsaw : 0.005044, class : 0.933165
Accuracies on test: jigsaw : 0.005

Accuracies on val: jigsaw : 0.021438, class : 0.938209
Accuracies on test: jigsaw : 0.012695, class : 0.630371
New epoch - lr: 0.001
0/55 of epoch 9/30 jigsaw : 3.509357, class : 0.181098 - acc jigsaw : 0.015625, class : 0.820312 [bs:128]
30/55 of epoch 9/30 jigsaw : 3.485747, class : 0.166843 - acc jigsaw : 0.039062, class : 0.726562 [bs:128]
Accuracies on val: jigsaw : 0.022699, class : 0.936948
Accuracies on test: jigsaw : 0.016602, class : 0.660156
New epoch - lr: 0.001
0/55 of epoch 10/30 jigsaw : 3.473294, class : 0.066021 - acc jigsaw : 0.031250, class : 0.828125 [bs:128]
30/55 of epoch 10/30 jigsaw : 3.471968, class : 0.115532 - acc jigsaw : 0.039062, class : 0.820312 [bs:128]
Accuracies on val: jigsaw : 0.018916, class : 0.935687
Accuracies on test: jigsaw : 0.017090, class : 0.635254
New epoch - lr: 0.001
0/55 of epoch 11/30 jigsaw : 3.507698, class : 0.045503 - acc jigsaw : 0.023438, class : 0.812500 [bs:128]
30/55 of epoch 11/30 jigsaw : 3.503085, class : 0.072675 - acc jig

0/53 of epoch 1/30 jigsaw : 3.426723, class : 2.025293 - acc jigsaw : 0.078125, class : 0.164062 [bs:128]
30/53 of epoch 1/30 jigsaw : 3.505164, class : 0.539317 - acc jigsaw : 0.039062, class : 0.710938 [bs:128]
Accuracies on val: jigsaw : 0.039318, class : 0.866317
Accuracies on test: jigsaw : 0.025597, class : 0.671502
New epoch - lr: 0.001
0/53 of epoch 2/30 jigsaw : 3.495488, class : 0.443493 - acc jigsaw : 0.046875, class : 0.695312 [bs:128]
30/53 of epoch 2/30 jigsaw : 3.466252, class : 0.307310 - acc jigsaw : 0.046875, class : 0.765625 [bs:128]
Accuracies on val: jigsaw : 0.034076, class : 0.882045
Accuracies on test: jigsaw : 0.019198, class : 0.676195
New epoch - lr: 0.001
0/53 of epoch 3/30 jigsaw : 3.440978, class : 0.335447 - acc jigsaw : 0.062500, class : 0.734375 [bs:128]
30/53 of epoch 3/30 jigsaw : 3.471107, class : 0.324651 - acc jigsaw : 0.007812, class : 0.781250 [bs:128]
Accuracies on val: jigsaw : 0.022280, class : 0.889908
Accuracies on test: jigsaw : 0.031997, c

...

Accuracies on val: jigsaw : 0.038554, class : 0.907229
Accuracies on test: jigsaw : 0.032335, class : 0.888024
New epoch - lr: 0.001
0/58 of epoch 11/30 jigsaw : 3.446863, class : 0.045660 - acc jigsaw : 0.093750, class : 0.703125 [bs:128]
30/58 of epoch 11/30 jigsaw : 3.504124, class : 0.103515 - acc jigsaw : 0.046875, class : 0.812500 [bs:128]
Accuracies on val: jigsaw : 0.038554, class : 0.907229
Accuracies on test: jigsaw : 0.029341, class : 0.886826
New epoch - lr: 0.001
0/58 of epoch 12/30 jigsaw : 3.449057, class : 0.101281 - acc jigsaw : 0.054688, class : 0.851562 [bs:128]
30/58 of epoch 12/30 jigsaw : 3.497599, class : 0.097407 - acc jigsaw : 0.039062, class : 0.742188 [bs:128]
Accuracies on val: jigsaw : 0.043373, class : 0.903614
Accuracies on test: jigsaw : 0.037725, class : 0.891617
New epoch - lr: 0.001
0/58 of epoch 13/30 jigsaw : 3.482166, class : 0.055331 - acc jigsaw : 0.031250, class : 0.828125 [bs:128]
30/58 of epoch 13/30 jigsaw : 3.457137, class : 0.176483 - acc j

0/42 of epoch 3/30 jigsaw : 3.543196, class : 0.339782 - acc jigsaw : 0.023438, class : 0.796875 [bs:128]
30/42 of epoch 3/30 jigsaw : 3.575436, class : 0.371016 - acc jigsaw : 0.007812, class : 0.773438 [bs:128]
Accuracies on val: jigsaw : 0.006612, class : 0.905785
Accuracies on test: jigsaw : 0.000000, class : 0.517943
New epoch - lr: 0.001
0/42 of epoch 4/30 jigsaw : 3.560302, class : 0.291273 - acc jigsaw : 0.007812, class : 0.750000 [bs:128]
30/42 of epoch 4/30 jigsaw : 3.522096, class : 0.397247 - acc jigsaw : 0.015625, class : 0.750000 [bs:128]
Accuracies on val: jigsaw : 0.003306, class : 0.895868
Accuracies on test: jigsaw : 0.000000, class : 0.496564
New epoch - lr: 0.001
0/42 of epoch 5/30 jigsaw : 3.521708, class : 0.113282 - acc jigsaw : 0.039062, class : 0.828125 [bs:128]
30/42 of epoch 5/30 jigsaw : 3.480464, class : 0.153787 - acc jigsaw : 0.023438, class : 0.859375 [bs:128]
Accuracies on val: jigsaw : 0.004959, class : 0.909091
Accuracies on test: jigsaw : 0.000000, c

Accuracies on val: jigsaw : 0.009917, class : 0.923967
Accuracies on test: jigsaw : 0.000000, class : 0.542886
New epoch - lr: 0.0001
0/42 of epoch 27/30 jigsaw : 3.535533, class : 0.015286 - acc jigsaw : 0.007812, class : 0.835938 [bs:128]
30/42 of epoch 27/30 jigsaw : 3.526454, class : 0.056930 - acc jigsaw : 0.031250, class : 0.812500 [bs:128]
Accuracies on val: jigsaw : 0.009917, class : 0.928926
Accuracies on test: jigsaw : 0.000000, class : 0.539323
New epoch - lr: 0.0001
0/42 of epoch 28/30 jigsaw : 3.534585, class : 0.074714 - acc jigsaw : 0.046875, class : 0.882812 [bs:128]
30/42 of epoch 28/30 jigsaw : 3.507553, class : 0.022849 - acc jigsaw : 0.015625, class : 0.875000 [bs:128]
Accuracies on val: jigsaw : 0.008264, class : 0.938843
Accuracies on test: jigsaw : 0.000000, class : 0.557648
New epoch - lr: 0.0001
0/42 of epoch 29/30 jigsaw : 3.529534, class : 0.064816 - acc jigsaw : 0.039062, class : 0.843750 [bs:128]
30/42 of epoch 29/30 jigsaw : 3.544695, class : 0.080806 - ac

0/42 of epoch 19/30 jigsaw : 3.515605, class : 0.010021 - acc jigsaw : 0.031250, class : 0.875000 [bs:128]
30/42 of epoch 19/30 jigsaw : 3.559328, class : 0.056814 - acc jigsaw : 0.007812, class : 0.828125 [bs:128]
Accuracies on val: jigsaw : 0.008264, class : 0.919008
Accuracies on test: jigsaw : 0.003818, class : 0.609061
New epoch - lr: 0.001
0/42 of epoch 20/30 jigsaw : 3.520378, class : 0.021405 - acc jigsaw : 0.031250, class : 0.843750 [bs:128]
30/42 of epoch 20/30 jigsaw : 3.548396, class : 0.090336 - acc jigsaw : 0.007812, class : 0.773438 [bs:128]
Accuracies on val: jigsaw : 0.008264, class : 0.912397
Accuracies on test: jigsaw : 0.022398, class : 0.530924
New epoch - lr: 0.001
0/42 of epoch 21/30 jigsaw : 3.538788, class : 0.068341 - acc jigsaw : 0.000000, class : 0.804688 [bs:128]
30/42 of epoch 21/30 jigsaw : 3.535929, class : 0.040678 - acc jigsaw : 0.007812, class : 0.882812 [bs:128]
Accuracies on val: jigsaw : 0.011570, class : 0.907438
Accuracies on test: jigsaw : 0.021

Accuracies on val: jigsaw : 0.128926, class : 0.897521
Accuracies on test: jigsaw : 0.099262, class : 0.641130
New epoch - lr: 0.001
0/42 of epoch 12/30 jigsaw : 3.383851, class : 0.072172 - acc jigsaw : 0.078125, class : 0.820312 [bs:128]
30/42 of epoch 12/30 jigsaw : 3.419147, class : 0.118875 - acc jigsaw : 0.054688, class : 0.820312 [bs:128]
Accuracies on val: jigsaw : 0.127273, class : 0.897521
Accuracies on test: jigsaw : 0.096971, class : 0.605498
New epoch - lr: 0.001
0/42 of epoch 13/30 jigsaw : 3.459158, class : 0.090059 - acc jigsaw : 0.031250, class : 0.757812 [bs:128]
30/42 of epoch 13/30 jigsaw : 3.433449, class : 0.022069 - acc jigsaw : 0.085938, class : 0.828125 [bs:128]
Accuracies on val: jigsaw : 0.114050, class : 0.899174
Accuracies on test: jigsaw : 0.083227, class : 0.554594
New epoch - lr: 0.001
0/42 of epoch 14/30 jigsaw : 3.450110, class : 0.046137 - acc jigsaw : 0.078125, class : 0.812500 [bs:128]
30/42 of epoch 14/30 jigsaw : 3.442685, class : 0.036842 - acc j

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jig_weight in [0.0]:
    print("\n======================\n%g\n===================" % args.jig_weight)
    for args.bias_whole_image in [1.0]: # [None, 0.01, 0.05, 0.1, 0.3, 0.5]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
dataloaders = {"train": data_helper.get_dataloader(args.source, args.jigsaw_n_classes, "train"),
           "val": data_helper.get_dataloader(args.target, args.jigsaw_n_classes, "val")}
dataset_sizes = {"train": len(dataloaders["train"].dataset),
                "val": len(dataloaders["val"].dataset)}
print(dataset_sizes)

In [ ]:
def get_optim_and_scheduler(network, epochs, lr):
    from torch import optim
    optimizer = optim.SGD(network.get_params(lr), weight_decay=.0005, momentum=.9, nesterov=True, lr=lr)
    step_size = int(epochs * .8)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)
    print(step_size)
    return optimizer, scheduler

model_ft = model_factory.get_network("caffenet")(jigsaw_classes=args.jigsaw_n_classes+1, classes=args.n_classes)
model_ft = model_ft.to(device)
# print(model_ft)

optimizer, scheduler = get_optim_and_scheduler(model_ft, args.epochs, args.learning_rate)

In [ ]:
def do_epoch(model, source, target, optimizer, logger, device):
    criterion = nn.CrossEntropyLoss()
    model.train()
    for it, ((data, jig_l, class_l), d_idx) in enumerate(source):
        data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
        
        optimizer.zero_grad()
        
        jigsaw_logit, class_logit = model(data)
        jigsaw_loss = criterion(jigsaw_logit, jig_l)
        class_loss = criterion(class_logit[d_idx!=target_id], class_l[d_idx!=target_id])
        _, cls_pred = class_logit.max(dim=1)
        _, jig_pred = jigsaw_logit.max(dim=1)
        loss = class_loss + jigsaw_loss * jig_weight
        
        loss.backward()
        optimizer.step()

        logger.log(it, len(source), {"jigsaw": jigsaw_loss.item(), "class": class_loss.item()},
                  {"jigsaw": torch.sum(jig_pred == jig_l.data).item(), "class":torch.sum(cls_pred == class_l.data).item()},
                  data.shape[0])
        del loss, class_loss, jigsaw_loss, jigsaw_logit, class_logit

    model.eval()
    with torch.no_grad():
        jigsaw_correct = 0
        class_correct = 0
        total = 0
        for it, ((data, jig_l, class_l), d_idx) in enumerate(target):
            data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
            jigsaw_logit, class_logit = model(data)
            _, cls_pred = class_logit.max(dim=1)
            _, jig_pred = jigsaw_logit.max(dim=1)
            class_correct += torch.sum(cls_pred == class_l.data)
            jigsaw_correct += torch.sum(jig_pred == jig_l.data)
            total += data.shape[0]
        logger.log_test({"jigsaw": float(jigsaw_correct) / total,
                         "class": float(class_correct) / total})


def do_training(args, model, source, target, optimizer, scheduler, device):
    logger = Logger(args)
    for k in range(args.epochs):
        scheduler.step()
        logger.new_epoch(scheduler.get_lr())
        do_epoch(model, source, target, optimizer, logger, device)
    return logger, model

In [ ]:
jig_weight = args.jig_weight
logger, model = do_training(args, model_ft, dataloaders["train"], dataloaders["val"], optimizer, scheduler, device)

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(args.source),args.target,args.epochs, jig_weight))

In [ ]:
def to_plt(inp):
    import numpy as np
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = model_ft.features[0] # models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(source),target,epochs, jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
for e,k in enumerate(range(0,l,int(updates))):
    print(k, logger.val_acc[e])

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from data.data_helper import get_val_dataloader, get_train_dataloader
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [ ]:
# loader = get_val_dataloader("photo",31,batch_size=10,multi=True)
loader, _ = get_train_dataloader(["photo"],31)

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(loader)
(tmp, v, c), d = next(iter_c)
print(tmp[:,0].shape)
for x in range(10):
#     image = tmp[0, x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()
    print(v[x])

# print(tmp[:,0].shape)
# for x in range(tmp.shape[1]):
# #     image = tmp[0, x]
#     image = torchvision.utils.make_grid(tmp[0, x],1,normalize=True)
#     plt.imshow(to_plt(image))
#     plt.show()
#     print(v[0,x])

    
# print(v.max(), v.min())

In [ ]:
res = []
for k in range(10):
    res.append(torch.rand(5, 100))
res = torch.stack(res,0)
res.mean(1).shape

In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
import torch
import random

In [ ]:
true = 0
for x in range(100):
    true += 0.1 > random.random()
true